# HOTEL HR PEOPLE ANALYTICS

### Hotel Analysis from a People Analytics perspective.
<p>This project will try to emulate HR analytics to continue practicing my analytical skills. Also, with this project, I want to <u>strengthen my knowledge</u> in <b><i>MySql</b></i> <b><i>and SQL</i></b>. I will use <b><i>Python</i></b> and <b><i>Power BI</i></b> for the data analysis.</p>
<p>I will start with doing a little of data engineering to design the databases I am going to work with. I will use <b><i>Figma</i></b> to draft the databases and the relationships between each other. Then I will fill the databases with the data, and later I will start the data analysis.</p>

### 1. Import libraries

In [1]:
# Libraries to manipulate the data
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
from app_pass import dbpass

# Library to deploy charts with the data
import seaborn as sns
import matplotlib.pyplot as plt

# Statmodels for predictions
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Connect to our MySQL database
import mysql.connector
from sqlalchemy import create_engine


# This is to ignore warnings.
import warnings
warnings.filterwarnings('ignore')

### 2. Working with our databases

<p>Previously, I created the databases I was going to work with. Because part of the data was difficult to create randomly, I downloaded the databases into an <i>.xlsx</i> file and worked with it.</p>

<p>I called the file <b><i>hotel_hranalytics.xlsx</i></b>, so now it's time to start working with it.</p>

In [31]:
# Let's load our databases
df_rawemp = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Employees', converters={'emp_id':str})

df_rawemp.head()

,emp_id,Name,Surname,Birthday,Age,Gender,on_license,hotel_id
0,3272,James,Smith,1957-08-09,67,M,0,FUESSP
1,3074,John,Johnson,1981-11-19,42,M,0,FUESSP
2,6627,Robert,Williams,1983-10-15,41,M,0,FUESSP
3,420,Michael,Brown,1976-04-05,48,M,0,FUESSP
4,4856,William,Jones,1968-11-20,55,M,0,FUESSP


In [32]:
df_rawht = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Hotels')

df_rawht.head()

,hotel_id,Name,Location,Opening,Stars,Budget
0,FUESSP,Sandy Shores Park,28 03 18.9N-14 19 21.4W,2001-03-05,4,350000000
1,TFNOBH,Ocean Breeze Haven,28 05 56.5N-16 44 54.6W,1998-10-05,5,550000000
2,ACECWR,Coral Wave Resort,28 51 25.9N-13 47 48.7 W,2000-05-05,5,480000000


In [33]:
df_rawhtcomp = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Hotel_Composition')

df_rawhtcomp.head()

,hc_id,Department,Active_employees,Emp_with_license,Total_employees,hotel_id
0,REFUESSP,Reception_Reservations,11,1,12,FUESSP
1,FLFUESSP,Floors_Laundry,35,3,38,FUESSP
2,KIFUESSP,Kitchen,35,3,38,FUESSP
3,BAFUESSP,Bar_Restaurant,31,7,38,FUESSP
4,ANFUESSP,Animation,11,1,12,FUESSP


In [34]:
df_rawempwages = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Employees_wages')

df_rawempwages.head()

,emp_wag_id,Price_$_Hour,Hours_worked,Work_overtime,Ovh$_75%,Gross_pay,Deductions_3%,Total_Payment,emp_id,hotel_id,hc_id,Payment_date
0,3272REFUESSP,14,129,4,10.50,1848.00,55.4400,1792.5600,3272,FUESSP,REFUESSP,2024-01-29
1,3074REFUESSP,14,143,3,10.50,2033.50,61.0050,1972.4950,3074,FUESSP,REFUESSP,2024-01-29
2,6627REFUESSP,18,135,4,13.50,2484.00,74.5200,2409.4800,6627,FUESSP,REFUESSP,2024-01-29
3,420REFUESSP,19,121,11,14.25,2455.75,73.6725,2382.0775,420,FUESSP,REFUESSP,2024-01-29
4,4856REFUESSP,14,132,7,10.50,1921.50,57.6450,1863.8550,4856,FUESSP,REFUESSP,2024-01-29


In [35]:
df_rawworkforce = pd.read_excel('../hotel_hranalytics/hotel_hranalytics.xlsx', sheet_name='Workforce_Composition')

df_rawworkforce.head()

,wkc_id,Department,Position,years_at_position,Entry_date,years_working,Staff,emp_id,hotel_id,hc_id
0,3272FUESSP,Reception_Reservations,Staff,1,2023-09-26,1,0,3272,FUESSP,REFUESSP
1,3074FUESSP,Reception_Reservations,Staff,1,2023-04-29,1,0,3074,FUESSP,REFUESSP
2,6627FUESSP,Reception_Reservations,3rd_Command,4,2014-01-17,10,7,6627,FUESSP,REFUESSP
3,420FUESSP,Reception_Reservations,3rd_Command,3,2012-10-25,12,7,420,FUESSP,REFUESSP
4,4856FUESSP,Reception_Reservations,Staff,1,2023-06-18,1,0,4856,FUESSP,REFUESSP


In [36]:
# If I needed let's put all together
df_combined = pd.concat([df_rawemp, df_rawht, df_rawhtcomp, df_rawworkforce, df_rawempwages])

df_combined.dtypes

emp_id                       object
Name                         object
Surname                      object
Birthday             datetime64[ns]
Age                         float64
Gender                       object
on_license                  float64
hotel_id                     object
Location                     object
Opening                      object
Stars                       float64
Budget                      float64
hc_id                        object
Department                   object
Active_employees            float64
Emp_with_license            float64
Total_employees             float64
wkc_id                       object
Position                     object
years_at_position           float64
Entry_date           datetime64[ns]
years_working               float64
Staff                       float64
emp_wag_id                   object
Price_$_Hour                float64
Hours_worked                float64
Work_overtime               float64
Ovh$_75%                    

In [37]:
# Employees Table
df_rawemp[['Age', 'on_license']].apply(pd.to_numeric)
df_rawemp[['hotel_id']].astype('str')

# Hotels Table
df_rawht['Stars'].apply(pd.to_numeric)
df_rawht[['hotel_id']].astype('str')
df_rawht.rename(columns={'Stars': 'Stars_type'}, inplace=True)

# Hotel Composition Table
df_rawhtcomp[['Active_employees', 'Emp_with_license', 'Total_employees']].apply(pd.to_numeric)
df_rawhtcomp[['hc_id', 'hotel_id']].astype('str')

# Workforce Composition Table
df_rawworkforce[['years_at_position', 'years_working', 'Staff']].apply(pd.to_numeric)
df_rawworkforce[['wkc_id', 'emp_id', 'hotel_id', 'hc_id']].astype('str')

# Employees Wages Table
df_rawempwages[['emp_wag_id', 'emp_id', 'hotel_id',	'hc_id']].astype('str')
df_rawempwages[['Price_$_Hour']].apply(pd.to_numeric)
df_rawempwages.rename(columns={'Ovh$_75%': 'Ovh$_75', 'Deductions_3%': 'Deductions_3'}, inplace=True)

df_rawemp.dtypes

emp_id                object
Name                  object
Surname               object
Birthday      datetime64[ns]
Age                    int64
Gender                object
on_license             int64
hotel_id              object
dtype: object

In [38]:
df_rawhtcomp.dtypes

hc_id               object
Department          object
Active_employees     int64
Emp_with_license     int64
Total_employees      int64
hotel_id            object
dtype: object

In [39]:
df_rawworkforce = df_rawworkforce.rename(columns={'Position': 'Positions'})
df_rawworkforce.dtypes


wkc_id                       object
Department                   object
Positions                    object
years_at_position             int64
Entry_date           datetime64[ns]
years_working                 int64
Staff                         int64
emp_id                        int64
hotel_id                     object
hc_id                        object
dtype: object

In [40]:
df_rawempwages.dtypes

emp_wag_id               object
Price_$_Hour              int64
Hours_worked              int64
Work_overtime             int64
Ovh$_75                 float64
Gross_pay               float64
Deductions_3            float64
Total_Payment           float64
emp_id                    int64
hotel_id                 object
hc_id                    object
Payment_date     datetime64[ns]
dtype: object

<p>Let's see our data through some visualizations. </p>

<p>The data type is now corrected and transformed. It's time to upload the data into <b><i>"hrhotelpa"</i></b> that is what our database is called  in MySQL. I will use <i>Python</i> to upload all the data to their corresponding table. Remember we have created the tables using <i>PopSQL</i>.</p>

<p>Let's start working with <b>MySQL</b>. We are required to connect to MySQL and later create a cursor to work with the queries. </p>

In [41]:
# Let's create the connection to MySQL
try:
    
    db = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = dbpass
    )
    print("Connection established")
    # Creating the cursor to execute queries
    cursor = db.cursor()
    
except mysql.connector.Error as err:
    print("An error occurred: ", err)


Connection established


In [42]:
# Creating a enginge connection
hostname = "localhost"
database = "hrhotelpa"
username = "root"
password = dbpass

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=database, user=username, pw=password))

In [43]:
# Add the databases to MySQL
# df_rawht.to_sql('Hotels', engine, if_exists='append', index=False)
# df_rawemp.to_sql('Employees', engine, if_exists='append', index=False)
# df_rawhtcomp.to_sql('Hotel_Composition', engine, if_exists='append', index=False)
# df_rawworkforce.to_sql('Workforce_Composition', engine, if_exists='append', index=False)
# df_rawempwages.to_sql('Employees_Wages', engine, if_exists='append', index=False)

<p>Now... that all the data have been uploaded to our MySQL database, it's time to close our connection.</p>

In [44]:
cursor.close()
db.close()

<p>Let's </i>.</p>